In [57]:
import librosa
import mir_eval
import mir_eval.display
import IPython.display

In [58]:
import numpy as np
import pandas as pd

import scipy.stats as stats
import matplotlib.pyplot as plot
import sklearn
import seaborn as sns

import pretty_midi

import pandas_profiling

#Path for the groove dataset

MIDI_PATH = "/home/mark/repos/Springboard/groove-v1.0.0-midionly/groove/"
DATA_PATH = "/home/mark/repos/Springboard/groove-v1.0.0-midionly/groove/info.csv"

groove_df = pd.read_csv(DATA_PATH)
groove_df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


In [34]:
pandas_profiling.ProfileReport(groove_df)

In [59]:
groove_df = groove_df[groove_df.beat_type != 'fill']
short = groove_df[groove_df.duration <=30]
short.count()
#43 < 5 seconds
#158 < 30 seconds
#261 < 1 minute

drummer           158
session           158
id                158
style             158
bpm               158
beat_type         158
time_signature    158
midi_filename     158
audio_filename    148
duration          158
split             158
dtype: int64

In [60]:
rock_df = groove_df.style
rock_df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid,drummer1/eval_session/1_funk-groove1_138_beat_4-4.wav,27.8723,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid,drummer1/eval_session/10_soul-groove10_102_beat_4-4.wav,37.6912,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid,drummer1/eval_session/2_funk-groove2_105_beat_4-4.wav,36.3512,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid,drummer1/eval_session/3_soul-groove3_86_beat_4-4.wav,44.7165,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid,drummer1/eval_session/4_soul-groove4_80_beat_4-4.wav,47.9875,test
5,drummer1,drummer1/eval_session,drummer1/eval_session/5,funk/groove5,84,beat,4-4,drummer1/eval_session/5_funk-groove5_84_beat_4-4.mid,drummer1/eval_session/5_funk-groove5_84_beat_4-4.wav,45.6875,test
6,drummer1,drummer1/eval_session,drummer1/eval_session/6,hiphop/groove6,87,beat,4-4,drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid,drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.wav,44.1192,test
7,drummer1,drummer1/eval_session,drummer1/eval_session/7,pop/groove7,138,beat,4-4,drummer1/eval_session/7_pop-groove7_138_beat_4-4.mid,drummer1/eval_session/7_pop-groove7_138_beat_4-4.wav,27.7065,test
8,drummer1,drummer1/eval_session,drummer1/eval_session/8,rock/groove8,65,beat,4-4,drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid,drummer1/eval_session/8_rock-groove8_65_beat_4-4.wav,59.0673,test
9,drummer1,drummer1/eval_session,drummer1/eval_session/9,soul/groove9,105,beat,4-4,drummer1/eval_session/9_soul-groove9_105_beat_4-4.mid,drummer1/eval_session/9_soul-groove9_105_beat_4-4.wav,36.5405,test


In [61]:
groove_df = groove_df[groove_df.duration > 30]
groove_df.count()
#Do we need audio files if we have the midi files??

drummer           345
session           345
id                345
style             345
bpm               345
beat_type         345
time_signature    345
midi_filename     345
audio_filename    296
duration          345
split             345
dtype: int64

In [62]:
groove_df = groove_df[groove_df.audio_filename.isna() == False]
groove_df.count()

drummer           296
session           296
id                296
style             296
bpm               296
beat_type         296
time_signature    296
midi_filename     296
audio_filename    296
duration          296
split             296
dtype: int64

In [63]:
sample = groove_df.iloc[0]
midi_sample = MIDI_PATH + sample.midi_filename
audio_sample = MIDI_PATH + sample.audio_filename
pm = pretty_midi.PrettyMIDI(midi_sample)

In [64]:
#midi_audio = pm.fluidsynth(audio_sample)

In [65]:
pr = pm.get_piano_roll()

#librosa.display.specshow(pr, y_axis='cqt_note', cmap=plot.cd.hot)

In [82]:
groove_df['style'].value_counts()
#rock_df = groove_df[groove_df.style == 'rock']
#groove_df[groove_df.style == 'rock']
r = "rock"
rock_df = groove_df.query('style=="'+r+'"')
rock_df
styles = groove_df['style'].value_counts()
style_durations = pd.DataFrame(columns=['style','max','min'])
for s in styles.index:
    style_df = groove_df.query('style=="'+s+'"')
    style_durations = style_durations.append({'style': s, 'max': style_df['duration'].max(), 
                                             'min': style_df['duration'].min()}, ignore_index=True)
style_durations.sort_values(by=['min'], ascending='False')

,style,max,min
24,latin/brazilian-baiao,227.155871,30.060481
0,rock,305.406445,30.209229
28,latin/brazilian-ijexa,92.421370,31.101877
5,soul,132.277344,31.902405
3,jazz,183.415349,31.915625
...,...,...,...
68,blues/shuffle,269.532878,269.532878
63,rock/folk,271.551525,271.551525
46,latin/dominican-merengue,307.425654,307.425654
62,rock/rockabilly,313.186250,313.186250
